In [1]:
from langchain_core.documents import Document

In [2]:
doc = Document(
    page_content="this is the main content i am using to create rag",
    metadata={
        "source": "exmaple.txt",
        "pages":1,
        "author":"Varun Bharadwaj S",
        "data_created":"2026-11-02"
    }
)
doc

Document(metadata={'source': 'exmaple.txt', 'pages': 1, 'author': 'Varun Bharadwaj S', 'data_created': '2026-11-02'}, page_content='this is the main content i am using to create rag')

In [3]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [4]:
sample_text={
    "../data/text_files/python_intro.txt":"""Python is a high-level, interpreted, general-purpose programming language. 
    Created by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. 
    It features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.
    Python's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation."""
}

for file_path,content in sample_text.items():
    with open(file_path,"w",encoding="utf-8") as f:
        f.write(content)
print("done creating file")

done creating file


In [5]:
from langchain_community.document_loaders import TextLoader


loader = TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

/Users/varunbharadwaj/vbsrag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="Python is a high-level, interpreted, general-purpose programming language. \n    Created by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. \n    It features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.\n    Python's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation.")]


In [6]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader("../data/text_files"
,glob="**/*.txt",
loader_cls= TextLoader,
loader_kwargs={
    "encoding":"utf-8"
},
show_progress=False

)

documents= dir_loader.load()
documents

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="Python is a high-level, interpreted, general-purpose programming language. \n    Created by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. \n    It features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.\n    Python's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation.")]

In [7]:
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader, PyPDFLoader


dir_loader = DirectoryLoader("../data/pdf"
,glob="**/*.pdf",
loader_cls= PyMuPDFLoader,
show_progress=False

)

pdf_documents= dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'macOS Version 26.2 (Build 25C56) Quartz PDFContext', 'creator': '', 'creationdate': "D:20260211092504Z00'00'", 'source': '../data/pdf/Rag Application and its use cases.pdf', 'file_path': '../data/pdf/Rag Application and its use cases.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20260211092504Z00'00'", 'trapped': '', 'modDate': "D:20260211092504Z00'00'", 'creationDate': "D:20260211092504Z00'00'", 'page': 0}, page_content='Rag Application and its use cases \n \n1. What is RAG? \n• Full form: retrieval argument, generation \n• It is the retrieval of relevant data from the source according to \nuser requirement \n• Referred to information outside of its training data source \n \n2. What is similarity search? \n• A type of search which composers the similarity using queries \n• Pardon is a technique to convert data into chunks or break them \ndown into diFerent parts \n• Embedding is a t

In [8]:
type(pdf_documents[0])


langchain_core.documents.base.Document

In [9]:
import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity



In [10]:
class EmbeddingManager:

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
         Initialize the embedding manager
         
         Args:
           model_name: HuggingFace model name for sentence embeddings
        """

        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """load the sentence transformer model"""

        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimensions: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
         texts: Lists of text strings to embed 

        Return:
         numpy array of embedding with shape(len(texts), embedding_dim)
         """

        if not self.model:
            raise ValueError("Model not found")

        print(f"Generating embeddings for {len(texts)} texts")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

# ✅ These lines should NOT be indented (they're outside the class)
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1449.46it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimensions: 384


Vector store 

In [11]:
import os
import numpy as np
import chromadb
import uuid
from typing import List, Any

In [12]:
class VectorStore:

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        
        self.collection_name = collection_name
        self.persist_directory = persist_directory  # ✅ Fixed typo
        self.client = None 
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        
        try:
            os.makedirs(self.persist_directory, exist_ok=True)  # ✅ Fixed: removed 'self,' and fixed variable name
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            self.collection = self.client.get_or_create_collection(  # ✅ Fixed: singular 'collection'
                name=self.collection_name,
                metadata={"description": "pdf doc embeddings for rag"}  # ✅ Fixed: metadata spelling
            )
            print(f"Vector store initialized successfully")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):  # ✅ Fixed: documents (plural)
        
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store")
        
        ids = []
        metadata_list = []  # ✅ Fixed: metadata_list instead of metdata
        document_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):  # ✅ Fixed: documents
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            doc_metadata = dict(doc.metadata)  # ✅ Fixed: metadata spelling
            doc_metadata['doc_index'] = i
            doc_metadata['content_length'] = len(doc.page_content)
            metadata_list.append(doc_metadata)  # ✅ Fixed: append to list, not dict
            
            document_text.append(doc.page_content)  # ✅ Fixed: variable name consistency
            
            embeddings_list.append(embedding.tolist())
        
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadata_list,  # ✅ Fixed: metadatas (plural) with correct list
                documents=document_text
            )
            print(f"Successfully added {len(documents)} documents")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error: {e}")
            raise


# ✅ CRITICAL: This must be OUTSIDE the class, with different variable name
vectorstore = VectorStore()  # Use lowercase 'vectorstore', not 'VectorStore'

Vector store initialized successfully


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Max characters per chunk
    chunk_overlap=50,      # Overlap between chunks to maintain context
    length_function=len,   # How to measure chunk size
    separators=["\n\n", "\n", " ", ""]  # How to split the text
)

# Split your PDF documents into chunks
chunks = text_splitter.split_documents(pdf_documents)

# Display the chunks
print(f"Total chunks created: {len(chunks)}")
print(f"\nFirst chunk:\n{chunks[0]}")

Total chunks created: 22

First chunk:
page_content='Rag Application and its use cases 
 
1. What is RAG? 
• Full form: retrieval argument, generation 
• It is the retrieval of relevant data from the source according to 
user requirement 
• Referred to information outside of its training data source 
 
2. What is similarity search? 
• A type of search which composers the similarity using queries 
• Pardon is a technique to convert data into chunks or break them 
down into diFerent parts' metadata={'producer': 'macOS Version 26.2 (Build 25C56) Quartz PDFContext', 'creator': '', 'creationdate': "D:20260211092504Z00'00'", 'source': '../data/pdf/Rag Application and its use cases.pdf', 'file_path': '../data/pdf/Rag Application and its use cases.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20260211092504Z00'00'", 'trapped': '', 'modDate': "D:20260211092504Z00'00'", 'creationDate': "D:20260211092504Z00'00'", 'page': 0}


In [14]:

chunks

[Document(metadata={'producer': 'macOS Version 26.2 (Build 25C56) Quartz PDFContext', 'creator': '', 'creationdate': "D:20260211092504Z00'00'", 'source': '../data/pdf/Rag Application and its use cases.pdf', 'file_path': '../data/pdf/Rag Application and its use cases.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20260211092504Z00'00'", 'trapped': '', 'modDate': "D:20260211092504Z00'00'", 'creationDate': "D:20260211092504Z00'00'", 'page': 0}, page_content='Rag Application and its use cases \n \n1. What is RAG? \n• Full form: retrieval argument, generation \n• It is the retrieval of relevant data from the source according to \nuser requirement \n• Referred to information outside of its training data source \n \n2. What is similarity search? \n• A type of search which composers the similarity using queries \n• Pardon is a technique to convert data into chunks or break them \ndown into diFerent parts'),
 Document(metadat

In [15]:
# Initialize the EmbeddingManager
embedding_manager = EmbeddingManager()

Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1534.25it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimensions: 384


In [16]:
# Extract text content from chunks
chunk_texts = [chunk.page_content for chunk in chunks]

# Generate embeddings using your EmbeddingManager
print("Generating embeddings for chunks...")
chunk_embeddings = embedding_manager.generate_embeddings(chunk_texts)

# Display results
print(f"\n✅ Successfully generated embeddings!")
print(f"Number of chunks: {len(chunks)}")
print(f"Embedding shape: {chunk_embeddings.shape}")
print(f"Each chunk is represented as a {chunk_embeddings.shape[1]}-dimensional vector")

# Show a sample
print(f"\nFirst chunk text (preview):")
print(chunk_texts[0][:200] + "...")
print(f"\nFirst chunk embedding (first 10 values):")
print(chunk_embeddings[0][:10])

Generating embeddings for chunks...
Generating embeddings for 22 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]

Generated embeddings with shape: (22, 384)

✅ Successfully generated embeddings!
Number of chunks: 22
Embedding shape: (22, 384)
Each chunk is represented as a 384-dimensional vector

First chunk text (preview):
Rag Application and its use cases 
 
1. What is RAG? 
• Full form: retrieval argument, generation 
• It is the retrieval of relevant data from the source according to 
user requirement 
• Referred to ...

First chunk embedding (first 10 values):
[-0.05452608  0.10618768 -0.02438059 -0.03262917 -0.0893824   0.04034872
  0.05058878  0.03790223 -0.02721749 -0.07658855]


In [17]:
class RAGRetriever:

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []


# Create the retriever instance (outside the class)
rag_retriever = RAGRetriever(vectorstore, embedding_manager)

In [18]:
rag_retriever

In [19]:
rag_retriever.retrieve("what is Cover letter?")

Retrieving documents for query: 'what is Cover letter?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.52it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

In [20]:
print(f"Documents in collection: {vectorstore.collection.count()}")

Documents in collection: 22


In [21]:
vectorstore.add_documents(chunks, chunk_embeddings)

Adding 22 documents to vector store
Successfully added 22 documents
Total documents in collection: 44


In [59]:
# Recreate the retriever to ensure it uses the correct vector store
rag_retriever = RAGRetriever(vectorstore, embedding_manager)

In [25]:
results = rag_retriever.retrieve('cretificates ?', top_k=5)
print(f"Found {len(results)} results")
for result in results:
    print(f"\nRank {result['rank']}: Score {result['similarity_score']:.3f}")
    print(f"Content preview: {result['content'][:200]}...")

Retrieving documents for query: 'cretificates ?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Found 0 results


In [23]:
rag_retriever.retrieve("Varun Bharadwaj ?")

Retrieving documents for query: 'Varun Bharadwaj ?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]